In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime

now = datetime.datetime.now().strftime("%Y%m%d")

In [2]:
#new chrome driver

def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    ban_notify=webdriver.ChromeOptions()
    prefs={"profile.default_content_setting_values.notifications" : 2}
    ban_notify.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=ban_notify)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [3]:
# 日期格式

def func(x):
    
    dot = x.split("・")[0]
    year = x.split("・")[1].split("年")[0]
    month = x.split("・")[1].split("年")[1].split("月")[0]
    day = x.split("・")[1].split("年")[1].split("月")[1].split("日")[0]
    if len(day)  == 3:
        day = day[0] + day[2]

        year = "".join(str(i) for i in year)
        month = "".join(str(i) for i in month)
        day = "".join(str(i) for i in day)
    if len(month) == 3:
        month = month[0] + month[2]
    if len(day) == 3:
        day = day[0] + day[2]
    final_date = year + "." + month + "." + day
    return final_date

In [13]:
#抓取貼文連結
def getPostLink(url,driver):
    driver.get(url)
    
    try:
        older_btn=driver.find_element(By.XPATH,'//*[@id="module-engadget-tag-streams"]/div[2]/div/div[2]/div[1]/ul/li[21]/button')
        ActionChains(driver).move_to_element(older_btn).perform()
        older_btn.click()
        time.sleep(0.5)
    except:
        print("No older Btn")
    
#     滾輪拉到底-> 才能抓全部  (有試拿掉但好像抓不到全部，這邊先留著)  
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
    post_link_list = []
    soup = BeautifulSoup(driver.page_source,"html.parser")

    for datee in soup.find_all("div", class_="Px(20px)--sm Py(15px)--sm Bgc(white)--sm W(100%)"):
        
        for l in datee.find_all("a", class_="C(engadgetFontBlack) Fw(400) Td(n) Bdbw(1px):h Bdbc(lightgrey):h Bdbs(s):h Fz(30px) Lh(35px) Fz(22px)--sm Lh(25px)--sm"):
            # 抓日期
            for d in datee.find_all("span", class_="C(engadgetFontLightGray) Mt(3px)"):
                temp = []
                temp.append(l.get_text())
                temp.append("https://chinese.engadget.com/"+l["href"])
                temp.append(d.get_text())
                post_link_list.append(temp)
                
    link_df = pd.DataFrame(data=post_link_list, columns=['Post Title', 'Link', 'Date'])
    
    link_df["Date"] = link_df["Date"].apply(func)
    link_df["Date"] = pd.to_datetime(link_df["Date"], format="%Y/%m/%d")
    
    FinishedCrawler(driver)
    return link_df

In [7]:
#抓取貼文留言

def getPostComment(url,driver):
    driver.get(url)
            
    #Get Comment       
    soup = BeautifulSoup(driver.page_source,"html.parser")
    content_all = str(soup.find("div",id="post-center-col"))
    content_html = content_all[:content_all.find('<div id="post-bottom-social-share">')]
    content_fix_soup = BeautifulSoup(content_html, "html.parser")
    content_text_all=[]
    for i in content_fix_soup.find_all("p"):
        if(i.find("a") != None):
            continue
        else:
            content_text_all.append(i.get_text())
    FinishedCrawler(driver)
    return content_text_all

In [5]:
link_df = getPostLink("https://chinese.engadget.com/tag/iphone%2013",OpenChromeDriver())
link_df

,Post Title,Link,Date
0,處處被看著？必備防窺配件推介,https://chinese.engadget.com//protect-privacy-...,2022-05-16
1,Apple 開始在印度生產 iPhone 13,https://chinese.engadget.com//apple-iphone-13-...,2022-04-12
2,Moment 推出售價 US$99 的 MagSafe 版 Mobile Filmmaker...,https://chinese.engadget.com//moments-mobile-f...,2022-03-30
3,綠色 iPhone 13 mini 和松嶺綠色 iPhone 13 Pro 真機實拍,https://chinese.engadget.com//iphone-13-green-...,2022-03-12
4,就是今晚！全新松嶺綠色 iPhone 13 Pro 開始接受預訂,https://chinese.engadget.com//apple-green-ipho...,2022-03-11
5,Apple 2022 首波新品總整理：Mac Studio、5G 版 iPhone SE、M...,https://chinese.engadget.com//apple-march-2022...,2022-03-09
6,松嶺綠 iPhone 13、13 Pro 登場！3 月 18 日開賣,https://chinese.engadget.com//apple-announce-g...,2022-03-09
7,蘋果可能已在 iPhone 13 上取消了內建的降噪功能,https://chinese.engadget.com//apple-has-report...,2022-01-17
8,新年換新機，入手紅色 iPhone 13 應節又行善,https://chinese.engadget.com//apple-iphone-pro...,2022-01-12
9,iPhone，15 歲生日快樂！,https://chinese.engadget.com//iphone-15-%E9%80...,2022-01-11


In [9]:
# 先把爬下來的主題跟網址存起來
file_name = "crawler_data/Engadget_iphone13_Link"+now+".csv"
link_df.to_csv(file_name, index=False, encoding="utf-8")

In [10]:
contents = []
for link in link_df.loc[:,"Link"]:
    contents.append(getPostComment(link,OpenChromeDriver()))

content_df = pd.DataFrame({'Content': contents})

In [11]:
all_df = link_df.join(content_df)
# 刪除空值(兩種)
all_df = all_df[all_df.Content.str.len()>0].reset_index(drop=True)
for i in range(len(all_df)):
    if len(all_df.Content[i][0])<1:
        all_df = all_df.drop(i)
all_df

,Post Title,Link,Date,Content
0,處處被看著？必備防窺配件推介,https://chinese.engadget.com//protect-privacy-...,2022-05-16,[Engadget 編輯團隊致力為你蒐羅優質產品及優惠價格，文章內部份連結商戶與 Engad...
1,Moment 推出售價 US$99 的 MagSafe 版 Mobile Filmmaker...,https://chinese.engadget.com//moments-mobile-f...,2022-03-30,[MagSafe 版 Mobile Filmmaker Cage 適用於 iPhone 12...
3,就是今晚！全新松嶺綠色 iPhone 13 Pro 開始接受預訂,https://chinese.engadget.com//apple-green-ipho...,2022-03-11,[Engadget 編輯團隊致力為你蒐羅優質產品及優惠價格，文章內部份連結商戶與 Engad...
4,Apple 2022 首波新品總整理：Mac Studio、5G 版 iPhone SE、M...,https://chinese.engadget.com//apple-march-2022...,2022-03-09,[早安！如果你錯過了昨晚 Apple「Peek performance」產品發佈會的話，可以...
5,松嶺綠 iPhone 13、13 Pro 登場！3 月 18 日開賣,https://chinese.engadget.com//apple-announce-g...,2022-03-09,[更新：新的松嶺綠色將於 3 月 11 日起接受預訂， 3 月 18 日開始發售，iPhon...
6,蘋果可能已在 iPhone 13 上取消了內建的降噪功能,https://chinese.engadget.com//apple-has-report...,2022-01-17,[iPhone 13 的消噪功能是屬於輔助功能之一，其用意是在使用手機的聽筒講電話時，用手機...
7,新年換新機，入手紅色 iPhone 13 應節又行善,https://chinese.engadget.com//apple-iphone-pro...,2022-01-12,[Engadget 編輯團隊致力為你蒐羅優質產品及優惠價格，文章內部份連結商戶與 Engad...
8,iPhone，15 歲生日快樂！,https://chinese.engadget.com//iphone-15-%E9%80...,2022-01-11,"[為了慶祝這個里程碑，讓我們一起來回顧過去的每一款機種，並且談談它們每一代的特色吧！, 一年..."
9,風格實用兼備！精選推介手機掛繩、手機帶,https://chinese.engadget.com//phone-strap-and-...,2021-12-29,[Engadget 編輯團隊致力為你蒐羅優質產品及優惠價格，文章內部份連結商戶與 Engad...
11,Apple 將提供 DIY 維修零件套件，讓你自行修好 iPhone 13、12 和 Mac 電腦,https://chinese.engadget.com//apple-announces-...,2021-11-17,[有追蹤 iFixit 拆機報告的大家，都大概知道 Apple 的產品為了緊湊性和善用每一吋...


In [12]:
file_name = "crawler_data/Engadget_iphone13_Data"+now+".csv"
all_df.to_csv(file_name, index=False, encoding="utf-8")